In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [2]:
from pyspark.context import SparkContext 
sc = SparkContext.getOrCreate()
from pyspark.sql.session import SparkSession
#sc = SparkContext.getOrCreate('local')
spark = SparkSession(sc)

In [16]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

In [3]:
import os
print os.getcwd()


C:\Users\bhargavic


In [4]:
import pandas as pd

In [7]:
##Toy Dataset code

df_rev = pd.read_csv("D:\Grad Notes\Search\Project\Data\yelp_dataset\dataset\\review.csv")

# lines = spark.read.csv(mpath+"review.csv").rdd
# parts = lines.map(lambda row: row.value.split("::"))

In [10]:
#print df_rev
df_rev = df_rev.sample(frac=0.4)
#ratings = spark.createDataFrame(df_rev)

In [11]:
df_new = df_rev[['user_id','business_id','stars']]
df_new = pd.DataFrame(df_new.groupby(['user_id','business_id']).agg('mean'))
df_new.reset_index(inplace=True)
df_new.head()

,user_id,business_id,stars
0,---1lKK3aKOuomHnwAkAow,5cbsjFtrntUAeUx51FaFTg,1
1,---1lKK3aKOuomHnwAkAow,DXlDzOcpdUE_F21tok0fgw,4
2,---1lKK3aKOuomHnwAkAow,GGCVNcBQ9WGviYNiaR8tBw,5
3,---1lKK3aKOuomHnwAkAow,NAuUGBenC_TJoj2c6y9rbg,5
4,---1lKK3aKOuomHnwAkAow,PIXl4WvAqjl2w9Lj81zXvQ,1


In [48]:
from functools import partial

ulist = set(df_new['user_id'].tolist())
blist = set(df_new['business_id'].tolist())
user_zip = dict(zip(ulist, xrange(len(ulist))))
bus_zip = dict(zip(blist, xrange(len(blist))))

def assign_id(val, zip_val):
    return zip_val[val]
    
df_new['user_id_int'] = df_new['user_id'].apply(partial(assign_id, zip_val= user_zip))

df_new['business_id_int'] = df_new['business_id'].apply(partial(assign_id, zip_val=bus_zip))

In [49]:
df_new.head()

,user_id,business_id,stars,user_id_int,business_id_int
0,---1lKK3aKOuomHnwAkAow,5cbsjFtrntUAeUx51FaFTg,1,141783,29022
1,---1lKK3aKOuomHnwAkAow,DXlDzOcpdUE_F21tok0fgw,4,141783,78728
2,---1lKK3aKOuomHnwAkAow,GGCVNcBQ9WGviYNiaR8tBw,5,141783,42424
3,---1lKK3aKOuomHnwAkAow,NAuUGBenC_TJoj2c6y9rbg,5,141783,73260
4,---1lKK3aKOuomHnwAkAow,PIXl4WvAqjl2w9Lj81zXvQ,1,141783,13462


In [50]:
ratings  = spark.createDataFrame(df_new)
print ratings

DataFrame[user_id: string, business_id: string, stars: bigint, user_id_int: bigint, business_id_int: bigint]


In [51]:
ratings.show(10)

+--------------------+--------------------+-----+-----------+---------------+
|             user_id|         business_id|stars|user_id_int|business_id_int|
+--------------------+--------------------+-----+-----------+---------------+
|---1lKK3aKOuomHnw...|5cbsjFtrntUAeUx51...|    1|     141783|          29022|
|---1lKK3aKOuomHnw...|DXlDzOcpdUE_F21to...|    4|     141783|          78728|
|---1lKK3aKOuomHnw...|GGCVNcBQ9WGviYNia...|    5|     141783|          42424|
|---1lKK3aKOuomHnw...|NAuUGBenC_TJoj2c6...|    5|     141783|          73260|
|---1lKK3aKOuomHnw...|PIXl4WvAqjl2w9Lj8...|    1|     141783|          13462|
|---1lKK3aKOuomHnw...|WOO81gScY3_VpaIfX...|    4|     141783|          75744|
|---1lKK3aKOuomHnw...|d7Jgj1h_MILumtsTl...|    5|     141783|          38589|
|---1lKK3aKOuomHnw...|dM8Yp8StA1NdusK5T...|    3|     141783|          65743|
|---1lKK3aKOuomHnw...|eRJA6L1Rqi-3ITevo...|    5|     141783|           3525|
|---1lKK3aKOuomHnw...|gVhX5xcnGjHE3mF2e...|    5|     141783|   

In [53]:

# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
#                                      rating=float(p[2]), timestamp=long(p[3])))
# ratings = spark.createDataFrame(ratingsRDD)

(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="user_id_int", itemCol="business_id_int", ratingCol="stars")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)


#evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars",
#                                predictionCol="prediction")



#rmse = evaluator.evaluate(predictions)
#print("Root-mean-square error = " + str(rmse))

# # Generate top 10 movie recommendations for each user
# userRecs = model.recommendForAllUsers(10)
# # Generate top 10 user recommendations for each movie
# movieRecs = model.recommendForAllItems(10)

In [55]:
predictions = predictions.dropna()
print predictions.take(10)

[Row(user_id=u'9PWGLiZgOPT0XloR-vms0w', business_id=u'e9q0RPoKoja0C4Q_a6cnZA', stars=3, user_id_int=154945, business_id_int=1238, prediction=0.031763311475515366), Row(user_id=u'_fpHwIiEJOp79qED9YSE7Q', business_id=u'pHDLRIG2Xu1_1mkGAbyEmw', stars=4, user_id_int=29281, business_id_int=1645, prediction=1.205211877822876), Row(user_id=u'_c_qAv00T9RIkYudBCy3NQ', business_id=u'pHDLRIG2Xu1_1mkGAbyEmw', stars=5, user_id_int=194412, business_id_int=1645, prediction=2.2061519622802734), Row(user_id=u'tu5AwWOh6QlSvpi5CXalQg', business_id=u'pHDLRIG2Xu1_1mkGAbyEmw', stars=2, user_id_int=105694, business_id_int=1645, prediction=-4.052732944488525), Row(user_id=u'szyiQXEhOZYmFwBtHGYoDQ', business_id=u'pHDLRIG2Xu1_1mkGAbyEmw', stars=3, user_id_int=57458, business_id_int=1645, prediction=0.44852501153945923), Row(user_id=u'N_LyFYrey_LuxJFkdFwRkw', business_id=u'pHDLRIG2Xu1_1mkGAbyEmw', stars=2, user_id_int=178308, business_id_int=1645, prediction=-2.8874943256378174), Row(user_id=u'W41iFVGdDatFi-C84g

In [56]:

evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars",
                               predictionCol="prediction")



rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 4.68695941334


In [60]:
type(predictions)

pyspark.sql.dataframe.DataFrame

In [63]:
predictions.agg({'prediction':'min'}).collect()[0][0]

-22.941612243652344